In [26]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

In [4]:
data = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")

In [8]:
data.tail()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.0,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.0,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.0,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.0,0.5,4.43,0.0,1.0,26.58,NaN,NaN
3066765,2,2023-01-31 23:07:32,2023-01-31 23:21:56,NaN,2.85,NaN,None,262,143,0,15.97,0.0,0.5,2.00,0.0,1.0,21.97,NaN,NaN


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

### Question 1 : Find total columns

In [7]:
data.shape

(3066766, 19)

### Question 2 : find the mean of duration

In [15]:
%%time
data['duration'] = data['tpep_dropoff_datetime'] - data['tpep_pickup_datetime'] # create a new column for duration
data['duration'] = data['duration'].apply(lambda x : x.total_seconds()/60)       # Converting duration to minutes

CPU times: user 8.04 s, sys: 949 ms, total: 8.99 s
Wall time: 8.97 s


In [16]:
data['duration'].describe() # mean is 42.59

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

### Question 3 : Dropping outliers

In [19]:
# remove records with duration greater than 60 and less than 1
# why? because it's requested in question. you can also try by
# using Inter Quantile Ranges or Z-Score to identify outliers.
#

df = data[(data["duration"] >= 1) & (data["duration"]<= 60)]

In [20]:
# left over records percentage

total_records = data.shape[0]
total_df = df.shape[0]

print(f"left over percentage : {(total_df/total_records)*100}")

left over percentage : 98.1220282212598


### Question 4 : One Hot Encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
- Fit a dictionary vectorizer
- Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [22]:
select_feats = ["PULocationID", "DOLocationID"]

df[select_feats] = df[select_feats].astype("str")

<ipython-input-22-cd5e38749a06>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[select_feats] = df[select_feats].astype("str")


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3009173 entries, 0 to 3066765
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           object        
 8   DOLocationID           object        
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee            floa

In [30]:
dv = DictVectorizer()

train_dicts = df[select_feats].to_dict(orient="records")

In [32]:
X_train = dv.fit_transform(train_dicts)

In [33]:
X_train.shape

(3009173, 515)

### question 5 : Training a Model

In [34]:
y_train = df['duration']

In [35]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [36]:
y_preds = model.predict(X_train)

In [42]:
from sklearn.metrics import mean_squared_error
import math

In [43]:
RMSE = math.sqrt(mean_squared_error(y_train, y_preds))

print(f"RMSE : {RMSE}")

RMSE : 7.649261929771859


### Question 6 : Evaluating the Model

validatation data set is on Feb dataset
same preprocessing which is done for jan is to be done again for Feb

In [53]:
val_data = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")
val_data.shape

(2913955, 19)

In [54]:
# set duration col

val_data["duration"] = val_data["tpep_dropoff_datetime"] - val_data["tpep_pickup_datetime"]
val_data["duration"] = val_data["duration"].apply(lambda x : x.total_seconds()/60)

In [55]:
val_data = val_data[(val_data["duration"] >= 1) & (val_data["duration"]<= 60)]

In [56]:
val_df = val_data[select_feats].astype("str")

val_dicts = val_df.to_dict(orient="records")

val_X = dv.transform(val_dicts)

In [57]:
print(f"val_x shape : {val_X.shape}")

val_x shape : (2855951, 515)


In [58]:
val_Y = val_data["duration"]

In [59]:
val_y_preds = model.predict(val_X)

val_RMSE = math.sqrt(mean_squared_error(val_Y, val_y_preds))
print(f"RMSE : {val_RMSE}")

RMSE : 7.811818933419717


### Revisit

+ why did we do outlier removal in validation dataset?
+ what is orient in to_dict function in dataframe?